In [1]:
import pandas as pd
from tqdm.notebook import tqdm, trange

In [5]:
DATASET_FILE = 'class_comments_pharo.csv'

df = pd.read_csv(DATASET_FILE)

In [6]:
df.dropna()

,AColorSelectorMorph,ColorComponentSelector showing an alpha gradient over a hatched background.
0,ASTCache,I am a simple cache for AST nodes correspondin...
1,ASTCacheReset,Announcement signaled to tell that the default...
3,ASTEvaluationTest,I am testing AST evaluation
4,ASTPluginMeaningOfLife,A simple example for a AST transforming compil...
5,ASTTransformExamplePluginActive,This class changes the default compiler used t...
...,...,...
7854,ZnUrl,"I am ZnUrl, an implementation of an interprete..."
7855,ZnUrlOperation,I am ZnUrlOperation.\rI am an abtract supercla...
7857,ZnUserAgentSession,I represent a session store for an HTTP client...
7859,ZnUtils,I am ZnUtils holding various utilities related...
